### Extract the Node IDs of the Hybrid III dummy

In [4]:
# Pick node IDs for HBM alone from key file

import numpy as np
import pandas as pd
# import xarray as xr
from qd.cae.dyna import KeyFile, D3plot

# open a keyfile
kf = KeyFile("LSTC.50thRigidFE.PostFeb28.WithBeltVehicleModelTemplate.k")
# kf = KeyFile("D:/Thesis/HBM_kinematics/Datasets_DOE/Shivzz/base_000/LSTC.50thRigidFE.PostFeb28.WithBeltVehicleModelTemplate.k")
nodes = kf["*NODE"][0]
temp = []
for line in nodes:
        if line.startswith("$")==False:
            temp.append(line)
# print(len(temp))
HBM_nodes=[]
for i in range(0,len(temp)-1):
        HBM_nodes.append(nodes[i].split(" ")[1])
print('Total no. of nodes in the Hybrid III model file: {}'.format(len(HBM_nodes)))

HBM_nodes = list(map(int, HBM_nodes))

Total no. of nodes in the Hybrid III model file: 7444


## Extract displacement data from D3plots 

In [10]:
import xarray as xr
from lasso.dyna import D3plot, ArrayType

### Extact node IDs in the full model (vehicle and dummy model node IDs) and
### timestate values at which results are written in D3plot

In [8]:

d3plot_lass = D3plot("D:/Thesis/HBM_kinematics/Datasets_DOE/Shivzz/base_001/d3plot") # Add path of any result file 
nodes_lass = d3plot_lass.arrays[ArrayType.node_ids]
Nd_lass = nodes_lass.tolist()

# d3plot_lass.arrays.keys()  # to find all the data read by lasso
times_lass = d3plot_lass.arrays["timesteps"] # Extract timestate values of recorded results



Initializations

In [9]:
folder_path = "D:/Thesis/HBM_kinematics/Datasets_DOE/Shivzz"   # path of D3plot reslts folders 
sims_n = 10                         # No. of simulation outputs to extract
fix_Nd = 3003092                    # Node ID of relative to which displacement 
                                    # of the Hybrid III dummy is calculated
ts = len(times_lass)                # No. of Timestates at which data is recorded in D3plots
Nd = HBM_nodes

# Remove defective nodes - associated with the vehicle and not related to HBM
Nd.remove(1009633)
Nd.remove(1009634)
Nd.remove(1009635)

C:\softwares\anaconda\anaconda3\envs\HBM\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Extract the relative displacement of the Hybrid III dummy with respect to the vehicle

In [11]:

samples = []
for i in range(0,sims_n,1):   
    
# add the D3plot file path of particular simulations
    sim_paths = "{}/base_{:03}/d3plot".format(folder_path, (i+1)) 
    
# use D3plot function from lasso.dyna toolkit to extrct the D3plot data
    d3plot_lass = D3plot(sim_paths)
    
# Extract nodal displacements for the full FE model (vehicle and dummy)...
#... - these are inculding the global position of the FE model
    coord_ts_lass = d3plot_lass.arrays[ArrayType.node_displacement] 
    
# Extract the inital coordinated of the FE model (vehicle and dummy)
    coord_lass = d3plot_lass.arrays[ArrayType.node_coordinates]     
    
# Subtract the initial coordinated of the model with global diplacement... 
#... values to get actual diplacement values
    disp_lass = np.subtract(coord_ts_lass, coord_lass)  
    
# save the displacement values in Xarray format for ease of handling i.e., ...
# to associate the raw nodal displacemnt values to calculate relative ...
#... displacements of Hybrid III dummy 
    disp_lass_xr = xr.DataArray(disp_lass,                                     
                       dims = ["times","nodes","DoFs"],                        
                        coords = {"times" : times_lass,
                                  "nodes" : Nd_lass,
                                  "DoFs"  : ['x_disp', 'y_disp', 'z_disp']
                                  })
    
# Subtract the displacement of the fixed node and the other displacments to get relative displacement    
    disp_lass_xr =  disp_lass_xr.sel(nodes=fix_Nd) - disp_lass_xr              
    disp_lass_xr = np.array(disp_lass_xr)
    
# append all displacement values of all simulations in a list
    samples.append(disp_lass_xr)

C:\softwares\anaconda\anaconda3\envs\HBM\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Save relative displacement outputs from all simulations in xarray format (vehcle and dummy)

In [18]:

samples_4D = np.array(samples)

samples_xarr = xr.DataArray(samples_4D,
                       dims = ["sims","times","DoFs","nodes"],
                        coords = { "sims" : list(range(sims_n)),
                                  "times" : times_lass,
                                  "DoFs"  : ['x_disp', 'y_disp', 'z_disp'],
                                  "nodes" : Nd_lass                                  
                                  })
# print(samples_xarr)


### Save only Hybrid III dummy nodal displacements

In [15]:
samples_xarr_HBM = samples_xarr.sel(nodes=Nd)

### Save only Hybrid III dummy nodal displacements in a file on PC

In [17]:
# samples_xarr_HBM.to_netcdf("samples_10_Dummy.nc")